<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-24 10:12:06
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  35.47 K
Current:  1.33 C
-------------------
Today PnL: 1.28 L (0.98%)
Current PnL: -16.48 L (-11.48%)
CY Booked + Current PnL: -10.11 L (-7.04%)
-------------------
Total profit:  3.29 L
Total loss:  -19.77 L
-------------------
Total Booked + Current PnL: 16.87 L (14.47%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.8%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.94 L (54.95%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LICI,827.06,983.0,-3.65,H-LC,87.10,97984.0,12797.0,3263.0,1.21,15.02,3.33,18.85,35.0,3.92,0.74,30.46,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.0,8.01,X-LC,63.44,160284.0,19475.0,6043.0,0.94,13.83,3.77,18.13,17.0,3.22,1.21,26.56,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,88.17,131868.0,-10611.0,11235.0,1.03,-7.45,8.52,0.44,242.0,-0.94,1.00,55.64,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-973.82,H-SC,98.92,83284.0,-12032.0,12068.0,-0.59,-12.62,14.49,0.04,153.0,-1.00,0.63,64.27,OX40N,NTT,MISC
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,49.46,99386.0,21024.0,13010.0,3.32,26.83,13.09,43.43,77.0,1.62,0.75,33.58,XY24,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,96.05,M-MC,56.99,223188.0,-1774.0,169422.0,2.38,-0.79,75.91,74.53,184.0,-0.01,1.69,29.58,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-33.09,M-MC,64.52,178562.0,-1246.0,113655.0,1.48,-0.69,63.65,62.52,170.0,-0.01,1.35,33.16,XR,NTT,BANKS
50,LALPATHLAB,2804.19,3545.00,-32.60,H-SC,35.48,155006.0,776.0,39976.0,0.51,0.50,25.79,26.42,126.0,0.02,1.17,22.24,X40N,NTT,HEALTHCARE
34,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,36.56,150644.0,-269.0,60725.0,1.20,-0.18,40.31,40.06,100.0,-0.00,1.14,23.22,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-36.43,H-SC,69.89,100907.0,403.0,30232.0,1.12,0.40,29.96,30.49,97.0,0.01,0.76,32.94,X5K,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HAPPSTMNDS,919.80,1496.95,-19.11,M-SC,78.49,68187.0,-28392.0,88991.0,4.57,-29.40,130.51,62.75,203.0,-0.32,0.52,19.56,AR,ATH,IT
48,JPPOWER,18.73,26.20,-33.95,L-SC,96.77,133891.0,-7951.0,64522.0,4.55,-5.61,48.19,39.88,258.0,-0.12,1.01,35.69,XY24,NTT,POWER
81,TITAGARH,1097.23,1548.00,-9.67,H-SC,40.86,158716.0,-33299.0,112180.0,3.54,-17.34,70.68,41.08,151.0,-0.30,1.20,34.33,XY24,NTT,ENGINEERING
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,49.46,99386.0,21024.0,13010.0,3.32,26.83,13.09,43.43,77.0,1.62,0.75,33.58,XY24,NTT,MISC
8,ATULAUTO,624.44,844.00,3348.65,M-SC,8.60,107439.0,-39929.0,91742.0,3.25,-27.09,85.39,35.16,230.0,-0.44,0.81,9.74,XY24,NTT,AUTO


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-30.12,M-SC,84.95,203639.0,32018.0,130044.0,-1.49,18.66,63.86,94.43,194.0,0.25,1.54,50.30,XR,NTT,DURABLES
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,93.55,69120.0,-44429.0,84478.0,-1.42,-39.13,122.22,35.27,260.0,-0.53,0.52,69.17,XR,NTT,HOTELS
78,TATAELXSI,7332.28,7332.00,-16.82,X-MC,34.41,87276.0,-15376.0,15369.0,-1.25,-14.98,17.61,-0.00,57.0,-1.00,0.66,32.67,OX40N,NTT,IT
55,MEDANTA,1064.65,1486.00,-1.06,X-MC,18.28,221525.0,16048.0,65283.0,-1.09,7.81,29.47,39.58,67.0,0.25,1.67,18.71,XY24,NTT,HEALTHCARE
64,ROUTE,1558.40,2249.26,-40.14,M-SC,58.06,64426.0,-38428.0,84024.0,-0.85,-37.36,130.42,44.33,188.0,-0.46,0.49,6.99,SR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,88.17,131868.0,-10611.0,11235.0,1.03,-7.45,8.52,0.44,242.0,-0.94,1.00,55.64,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-973.82,H-SC,98.92,83284.0,-12032.0,12068.0,-0.59,-12.62,14.49,0.04,153.0,-1.00,0.63,64.27,OX40N,NTT,MISC
78,TATAELXSI,7332.28,7332.0,-16.82,X-MC,34.41,87276.0,-15376.0,15369.0,-1.25,-14.98,17.61,-0.00,57.0,-1.00,0.66,32.67,OX40N,NTT,IT
42,INDIGOPNTS,1407.73,1408.0,120.42,H-SC,91.40,144609.0,-29950.0,29977.0,0.43,-17.16,20.73,0.02,127.0,-1.00,1.09,25.40,OX40N,NTT,PAINTS
76,SYMPHONY,1306.42,1306.0,-34.12,M-SC,1.08,140524.0,-30617.0,30564.0,0.25,-17.89,21.75,-0.03,190.0,-1.00,1.06,1.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,INFY,1461.46,2275.00,-10.98,X-LC,62.37,285917.0,24316.0,121315.0,0.84,9.29,42.43,55.67,1.0,0.20,2.16,15.97,X40,BTT,IT
80,TCS,3794.03,4998.00,-17.76,X-LC,30.11,267641.0,-28293.0,122205.0,1.11,-9.56,45.66,31.73,2.0,-0.23,2.02,7.13,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.40,X-LC,24.73,256581.0,-13766.0,48391.0,0.69,-5.09,18.86,12.81,5.0,-0.28,1.94,7.31,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,12.14,X-LC,53.76,256933.0,27700.0,39593.0,0.29,12.08,15.41,29.35,7.0,0.70,1.94,22.08,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-12.85,X-LC,22.58,276916.0,11490.0,45414.0,0.33,4.33,16.40,21.44,10.0,0.25,2.09,10.79,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.76,M-LC,49.46,99386.0,21024.0,13010.0,3.32,26.83,13.09,43.43,77.0,1.62,0.75,33.58,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,6.22,H-SC,82.80,119482.0,17478.0,27541.0,1.52,17.13,23.05,44.14,125.0,0.63,0.90,36.16,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,49.46,99386.0,21024.0,13010.0,3.32,26.83,13.09,43.43,77.0,1.62,0.75,33.58,XY24,NTT,MISC
92,WIPRO,243.46,420.00,-3.36,M-LC,80.65,165974.0,15029.0,94423.0,1.69,9.96,56.89,72.51,68.0,0.16,1.25,14.33,XR,NTT,IT
85,UNIONBANK,123.87,163.00,-4.91,M-LC,72.04,166377.0,25537.0,18950.0,2.44,18.13,11.39,31.59,86.0,1.35,1.26,44.52,XY24,NTT,BANKS
39,ICICIGI,1839.64,2260.25,-10.87,H-MC,83.87,146779.0,10646.0,20476.0,0.55,7.82,13.95,22.86,34.0,0.52,1.11,21.42,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-30.12,M-SC,84.95,203639.0,32018.0,130044.0,-1.49,18.66,63.86,94.43,194.0,0.25,1.54,50.30,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-3.36,M-LC,80.65,165974.0,15029.0,94423.0,1.69,9.96,56.89,72.51,68.0,0.16,1.25,14.33,XR,NTT,IT
41,INDIAMART,2327.09,4871.06,-48.11,H-SC,89.25,134975.0,11639.0,123192.0,1.41,9.44,91.27,109.32,122.0,0.09,1.02,33.35,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-36.43,H-SC,69.89,100907.0,403.0,30232.0,1.12,0.40,29.96,30.49,97.0,0.01,0.76,32.94,X5K,ATH,METALS
34,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,36.56,150644.0,-269.0,60725.0,1.20,-0.18,40.31,40.06,100.0,-0.00,1.14,23.22,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,93.55,69120.0,-44429.0,84478.0,-1.42,-39.13,122.22,35.27,260.0,-0.53,0.52,69.17,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7450.00,M-SC,100.00,82850.0,-10960.0,87581.0,1.37,-11.68,105.71,81.67,238.0,-0.13,0.63,60.44,XR,NTT,CERAMICS
54,MASFIN,326.60,402.39,-22.76,H-SC,45.16,87780.0,-10200.0,32935.0,0.76,-10.41,37.52,23.21,141.0,-0.31,0.66,27.31,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,1.03,X-SC,97.85,94058.0,-6905.0,101432.0,1.76,-6.84,107.84,93.63,60.0,-0.07,0.71,40.10,SR,ATH,CHEMICALS
52,LICI,827.06,983.00,-3.65,H-LC,87.10,97984.0,12797.0,3263.0,1.21,15.02,3.33,18.85,35.0,3.92,0.74,30.46,XY25,NTT,INSURANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7450.00,M-SC,100.00,82850.0,-10960.0,87581.0,1.37,-11.68,105.71,81.67,238.0,-0.13,0.63,60.44,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-973.82,H-SC,98.92,83284.0,-12032.0,12068.0,-0.59,-12.62,14.49,0.04,153.0,-1.00,0.63,64.27,OX40N,NTT,MISC
4,ALKYLAMINE,2347.98,4546.37,1.03,X-SC,97.85,94058.0,-6905.0,101432.0,1.76,-6.84,107.84,93.63,60.0,-0.07,0.71,40.10,SR,ATH,CHEMICALS
48,JPPOWER,18.73,26.20,-33.95,L-SC,96.77,133891.0,-7951.0,64522.0,4.55,-5.61,48.19,39.88,258.0,-0.12,1.01,35.69,XY24,NTT,POWER
69,SFL,1016.35,1287.00,328.99,H-SC,95.70,193655.0,-68563.0,138386.0,0.56,-26.15,71.46,26.63,145.0,-0.50,1.46,20.40,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,42.61
2,50,71.35


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.61
LC    38.08
MC    21.63
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.19
XY25     14.11
X40      12.48
X40N     10.77
XR       10.35
OX40N     8.30
AR        7.94
X5K       2.24
X200      1.45
SR        1.20
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.45
X-LC    15.41
M-SC    13.98
H-LC    13.24
H-MC     8.49
M-LC     8.33
X-MC     7.55
M-MC     5.59
X-SC     2.76
L-SC     2.42
L-LC     1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.42,-2.52,36.79
IT,12.07,-11.12,66.28
BANKS,8.01,-7.69,51.63
FINANCE,7.74,-17.55,60.10
MISC,6.71,-21.72,69.31
PAINTS,5.61,-17.61,40.79
ELECTRICAL,5.09,-4.47,49.71
HEALTHCARE,4.60,-5.81,36.72
AUTO,4.46,-14.53,64.89


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2803600.0,25
XR,1090309.0,13
AR,1021627.0,9
X40,667688.0,9
XY25,517624.0,10
X40N,504490.0,10
OX40N,332132.0,11
SR,185456.0,2
X5K,102775.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1922917.0,19
M-SC,1478598.0,18
X-LC,624622.0,9
M-MC,603873.0,5
H-LC,603622.0,12
H-MC,499867.0,7
X-MC,416362.0,6
M-LC,341642.0,6
L-SC,276715.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,889225.0,6
M-SC,XY24,598080.0,5
H-SC,AR,521604.0,3
M-MC,XY24,398747.0,3
M-SC,XR,334541.0,4
X-LC,X40,321993.0,3
H-MC,XY24,238973.0,2
H-LC,X40N,236734.0,5
M-LC,XY24,231283.0,4


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
